In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm import tqdm

In [2]:
# Trnasformaciones
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

In [3]:
# Cargar datos
train_data = datasets.ImageFolder(root='./train', transform=transform)
test_data = datasets.ImageFolder(root='./test', transform=transform)
validation_data = datasets.ImageFolder(root='./valid', transform=transform)

In [4]:
# Cargadores
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_data, batch_size=32, shuffle=False)

In [5]:
# Arquitectura
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 11)  

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 6 * 6)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
# Instancia del modelo
model = CNN()

In [7]:
# Función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Entrenamiento
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

100%|██████████| 4/4 [00:00<00:00,  4.59it/s]


Epoch 1/10, Loss: 2.4102863073349


100%|██████████| 4/4 [00:00<00:00,  4.98it/s]


Epoch 2/10, Loss: 2.344028651714325


100%|██████████| 4/4 [00:00<00:00,  5.01it/s]


...

Epoch 8/10, Loss: 1.1145468354225159


100%|██████████| 4/4 [00:00<00:00,  5.26it/s]


Epoch 9/10, Loss: 0.9376245141029358


100%|██████████| 4/4 [00:00<00:00,  5.23it/s]

Epoch 10/10, Loss: 0.9030008763074875


In [9]:

# Evaluación del modelo con conjunto de test
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")


Test Accuracy: 0.32727272727272727
